# Redes neuronales con Pytorch y Tensorflow


PyTorch y TensorFlow son bibliotecas de código abierto ampliamente utilizadas para la computación numérica y el desarrollo de modelos de aprendizaje profundo. PyTorch, desarrollado por Facebook's AI Research lab (FAIR), y TensorFlow, creado por Google Brain, ofrecen potentes herramientas para el desarrollo y entrenamiento de redes neuronales.

Ambas bibliotecas tienen como característica central el uso de tensores, que son estructuras de datos multidimensionales utilizadas para almacenar y manipular datos de manera eficiente. Los tensores en PyTorch y TensorFlow son similares a los arreglos de NumPy, pero tienen la ventaja de poder realizar operaciones en GPUs (Unidades de Procesamiento Gráfico), lo que permite una aceleración significativa en tareas computacionales intensivas, como el entrenamiento de redes neuronales.

Mientras que PyTorch se destaca por su flexibilidad y una sintaxis más intuitiva, que facilita la depuración y el desarrollo rápido de modelos, TensorFlow es conocido por su robustez en producción, su capacidad para escalar, y por ofrecer TensorFlow Serving para implementar modelos en producción.

## Pytorch



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

### Manejo de tensores

In [ ]:
list_a = [1,2,3,4]

In [ ]:
tensor_a = torch.tensor(list_a)

In [ ]:
tensor_a

In [ ]:
array_a = [[1, 1, 1],
       [2, 3, 4],
       [4, 5, 6]]

In [ ]:
array_b = [[7, 5, 4],
       [2, 2, 8],
       [6, 3, 8]]

In [ ]:
tensor_a = torch.tensor(array_a)
tensor_b = torch.tensor(array_b)

In [ ]:
tensor_c = tensor_a -tensor_b

tensor_d = tensor_a * tensor_b

tensor_e = tensor_c + tensor_d
print(tensor_e)

### Manejo de capas

`torch.nn` es un módulo fundamental de PyTorch que proporciona herramientas y clases para construir y entrenar redes neuronales de manera eficiente. Este módulo simplifica el proceso de definición, implementación y entrenamiento de modelos de aprendizaje profundo al ofrecer una variedad de componentes predefinidos que se pueden combinar para crear arquitecturas de redes complejas.


In [ ]:
input_tensor = torch.Tensor([[2, 3, 6, 7, 9, 3, 2, 1]])

linear_layer = nn.Linear(in_features=8, out_features=3)

In [ ]:
linear_layer(input_tensor)

Con nn.Sequential podemos crear modelos completos

In [ ]:
model = nn.Sequential(nn.Linear(8, 8),
                      nn.Linear(8, 1)
                     )

output = model(input_tensor)
print(output)

In [ ]:
model = nn.Sequential(nn.Linear(8, 5),
                      nn.Linear(5, 1)
                     )

output = model(input_tensor)
print(output)

### Funciones de activación

In [ ]:
input_tensor = torch.tensor([[0.8]])

sigmoid = nn.Sigmoid()
probability = sigmoid(input_tensor)
print(probability)

In [ ]:
input_tensor = torch.Tensor([[3, 4, 6, 2, 3, 6, 8, 9]])

model = nn.Sequential(
  nn.Linear(8,1),
  nn.Sigmoid()
)

output = model(input_tensor)
print(output)

### Ejemplo con dataset vinos

In [ ]:
data = pd.read_csv('winequality-red.csv')

In [ ]:
X = data.drop(columns=['quality'])
y = data['quality']

# Estandarizar las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y_encoded, dtype=torch.long)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

In [ ]:
input_size = X_train.shape[1]
num_classes =  len(label_encoder.classes_)

In [ ]:
input_size, num_classes

In [ ]:
model = nn.Sequential(
    nn.Linear(input_size, 64),  # Capa oculta 1
    nn.ReLU(),                  # Función de activación ReLU
    nn.Linear(64, 32),          # Capa oculta 2
    nn.ReLU(),                  # Función de activación ReLU
    nn.Linear(32, num_classes),  # Capa de salida
    nn.Softmax(dim=-1)
)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 1000
model.train()
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)

    # Calcular la pérdida
    loss = criterion(outputs, y_train)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 5 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
model.eval()  # Poner el modelo en modo de evaluación
with torch.no_grad():  # No calcular gradientes
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)  # Obtener las predicciones
    f1_micro = f1_score(y_test.numpy(), predicted.numpy(), average='micro')  # Calcular F1 micro

In [ ]:
print(f'F1 Micro: {f1_micro:.4f}')

In [ ]:
model = nn.Sequential(
    nn.Linear(input_size, 64),
    nn.BatchNorm1d(64),         # Batch Normalization
    nn.ReLU(),
    nn.Dropout(p=0.3),         # Dropout layer

    nn.Linear(64, 32),
    nn.BatchNorm1d(32),         # Batch Normalization
    nn.ReLU(),
    nn.Dropout(p=0.3),         # Dropout layer

    nn.Linear(32, 16),
    nn.BatchNorm1d(16),          # Batch Normalization
    nn.LeakyReLU(negative_slope=0.01),
    nn.Dropout(p=0.3),         # Dropout layer

    nn.Linear(16, num_classes),
    nn.Softmax(dim=-1)         # Softmax activation
)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 10000
model.train()
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)

    # Calcular la pérdida
    loss = criterion(outputs, y_train)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 5 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
model.eval()  # Poner el modelo en modo de evaluación
with torch.no_grad():  # No calcular gradientes
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)  # Obtener las predicciones
    f1_micro = f1_score(y_test.numpy(), predicted.numpy(), average='micro')  # Calcular F1 micro

In [ ]:
print(f'F1 Micro: {f1_micro:.4f}')

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
dataset = TensorDataset(X_train, y_train)

In [ ]:
batch_size = 300
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
model = nn.Sequential(
    nn.Linear(input_size, 64),
    nn.BatchNorm1d(64),         # Batch Normalization
    nn.ReLU(),
    nn.Dropout(p=0.2),         # Dropout layer

    nn.Linear(64, 32),
    nn.BatchNorm1d(32),         # Batch Normalization
    nn.ReLU(),                          # Dropout layer

    nn.Linear(32, 16),
    nn.BatchNorm1d(16),          # Batch Normalization
    nn.LeakyReLU(negative_slope=0.01),        # Dropout layer

    nn.Linear(16, num_classes),
    nn.Softmax(dim=-1)         # Softmax activation
)

In [ ]:
learning_rate = 0.001

# Definir el optimizador y la función de pérdida
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,  weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()


In [ ]:
num_epochs = 300

# Entrenar el modelo
for epoch in range(num_epochs):
    model.train()  # Cambiar el modelo a modo de entrenamiento
    running_loss = 0.0

    for batch_idx, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()  # Limpiar los gradientes
        outputs = model(inputs)  # Pasar los datos por el modelo
        loss = criterion(outputs, labels)  # Calcular la pérdida
        loss.backward()  # Calcular los gradientes
        optimizer.step()  # Actualizar los parámetros

        running_loss += loss.item()

    # Imprimir el promedio de pérdida de cada época
    avg_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

In [ ]:
model.eval()  # Poner el modelo en modo de evaluación
with torch.no_grad():  # No calcular gradientes
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)  # Obtener las predicciones
    f1_micro = f1_score(y_test.numpy(), predicted.numpy(), average='micro')

In [ ]:
print(f'F1 Micro: {f1_micro:.4f}')

https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/01-basics/feedforward_neural_network/main.py

## Tensorflow

En este cuaderno, utilizaremos TensorFlow para abordar un problema de regresión, prediciendo la eficiencia del consumo de combustible de automóviles (medida en millas por galón, MPG) utilizando el conjunto de datos Auto MPG. Este dataset proviene de la base de datos de aprendizaje automático del UCI Machine Learning Repository y contiene diversas características técnicas de autos, como el número de cilindros, desplazamiento del motor, potencia, y peso.

In [ ]:
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
dataset_path = keras.utils.get_file("auto-mpg.data", "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
dataset_path

In [ ]:
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model Year', 'Origin']
raw_dataset = pd.read_csv(dataset_path, names=column_names,
                      na_values = "?", comment='\t',
                      sep=" ", skipinitialspace=True)

dataset = raw_dataset.copy()
dataset.tail()

In [ ]:
dataset = dataset.dropna()

In [ ]:
origin = dataset.pop('Origin')

In [ ]:
dataset['USA'] = (origin == 1)*1.0
dataset['Europe'] = (origin == 2)*1.0
dataset['Japan'] = (origin == 3)*1.0
dataset.tail()

In [ ]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
sns.pairplot(train_dataset[["MPG", "Cylinders", "Displacement", "Weight"]], diag_kind="kde")

In [ ]:
train_stats = train_dataset.describe()
train_stats.pop("MPG")
train_stats = train_stats.transpose()
train_stats

In [ ]:
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

In [ ]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']

normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

In [ ]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

In [ ]:
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 1000

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()


In [ ]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [MPG]')
  plt.plot(hist['epoch'], hist['mae'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Val Error')
  plt.ylim([0,5])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$MPG^2$]')
  plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Val Error')
  plt.ylim([0,20])
  plt.legend()
  plt.show()


plot_history(history)

In [ ]:
model = build_model()

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(normed_train_data, train_labels, epochs=EPOCHS,
                    validation_split = 0.2, verbose=0, callbacks=[early_stop, PrintDot()])

plot_history(history)

In [ ]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} MPG".format(mae))